In [1]:
%pip install python-dotenv langchain langchain-openai langchain-community docx2txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
load_dotenv()

True

In [ ]:
embedding = OpenAIEmbeddings( model="text-embedding-3-large")
index_name = 'tax-index'
vectorstore = PineconeVectorStore.from_existing_index(
    index_name = index_name,
    embedding = embedding,
)

llm = ChatOpenAI(model='gpt-4o')
prompt = hub.pull("rlm/rag-prompt")
retriever = vectorstore.as_retriever(search_kwargs={'k':3})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={'prompt':prompt}
)
dictionary = ['사람을 나타내는 표현 -> 거주자']

prompt2 = ChatPromptTemplate.from_template(f"""
                                           사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을
                                           변경해 주세요. 만약 변경할 필요가 없다면 사용자의 질문을
                                           변경하지 않아도 됩니다.
                                           사전 : {dictionary}
                                           질문 : {{question}}
                                           """)

dictionary_chain = prompt2 | llm | StrOutputParser()
tax_chain = {"query" : dictionary_chain} | qa_chain

In [18]:
#query = "연봉 5천만원 거주자의 종합 소득세는 얼마인가?"
query = "연봉 5천만원 직장인 소득세는 얼마인가?"

In [20]:
retriever = vectorstore.as_retriever(search_kwargs={'k':3})
retriever.invoke(query)

[Document(id='dd2ca49a-7a6a-4d44-9e55-819b1c9759c6', metadata={'source': 'c:\\workspace\\tax_work\\data\\tax.docx'}, page_content='나. 그 밖의 배당소득에 대해서는 100분의 14\n\n3. 원천징수대상 사업소득에 대해서는 100분의 3. 다만, 외국인 직업운동가가 한국표준산업분류에 따른 스포츠 클럽 운영업 중 프로스포츠구단과의 계약(계약기간이 3년 이하인 경우로 한정한다)에 따라 용역을 제공하고 받는 소득에 대해서는 100분의 20으로 한다.\n\n4. 근로소득에 대해서는 기본세율. 다만, 일용근로자의 근로소득에 대해서는 100분의 6으로 한다.\n\n5. 공적연금소득에 대해서는 기본세율\n\n5의2.제20조의3제1항제2호나목 및 다목에 따른 연금계좌 납입액이나 운용실적에 따라 증가된 금액을 연금수령한 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 각 목의 요건을 동시에 충족하는 때에는 낮은 세율을 적용한다.\n\n가. 연금소득자의 나이에 따른 다음의 세율\n\n\n\n나. 삭제<2014. 12. 23.>\n\n다. 사망할 때까지 연금수령하는 대통령령으로 정하는 종신계약에 따라 받는 연금소득에 대해서는 100분의 4\n\n5의3. 제20조의3제1항제2호가목에 따라 퇴직소득을 연금수령하는 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 연금 실제 수령연차 및 연금외수령 원천징수세율의 구체적인 내용은 대통령령으로 정한다.\n\n가. 연금 실제 수령연차가 10년 이하인 경우: 연금외수령 원천징수세율의 100분의 70\n\n나. 연금 실제 수령연차가 10년을 초과하는 경우: 연금외수령 원천징수세율의 100분의 60\n\n6. 기타소득에 대해서는 다음에 규정하는 세율. 다만, 제8호를 적용받는 경우는 제외한다.\n\n가. 제14조제3항제8호라목 및 마목에 해당하는 소득금액이 3억원을 초과하는 경우 그 초과하는 분에 대해서는 100분의 

In [ ]:

prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [ ]:
ai_message = qa_chain.invoke({"query": query})


In [25]:
ai_message

{'query': '연봉 5천만원 직장인 소득세는 얼마인가?',
 'result': '연봉 5천만 원의 직장인의 소득세는 정확한 계산을 위해 기본세율과 각종 공제 항목을 고려해야 합니다. 일반적으로 한국의 소득세 기본세율은 구간에 따라 6%에서 45%까지이며, 연봉의 비과세 항목 및 세액 공제 여부에 따라 달라질 수 있습니다. 구체적인 소득세 금액은 근로소득 공제, 세액 공제 등 여러 세부사항을 반영해야 정확히 산출됩니다.'}

In [ ]:



#



In [ ]:
# 질문이 바뀌나 테스트
new_question = dictionary_chain.invoke({"question" : query})
new_question

'질문: 연봉 5천만원 거주자의 소득세는 얼마인가?'

In [40]:
ai_message = tax_chain.invoke({"question": query})

In [44]:
ai_message

{'query': '질문 : 연봉 5천만원 거주자 소득세는 얼마인가?',
 'result': '연봉 5천만 원의 소득세는 1,400만 원을 초과한 부분에 15%를 적용한 세액에 84만 원을 더하는 방식으로 계산됩니다. 따라서 소득세는 84만 원 + (5,000만 원 - 1,400만 원) * 0.15로 계산되어, 결과적으로 624만 원이 됩니다.'}